# Fifth sweep Neural network for classification of contaminants with MAT

2023-11-06

## 1. Formulate/outline the problem: classification

Simple neural network for classification of the contaminants using MAT transcriptomes


## 2. Data preprocessing

In [11]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
import wandb
import tensorflow as tf
from tensorflow import keras

In [12]:
import random
from sklearn.base import BaseEstimator, TransformerMixin


def select_samples(name, n=None, sample_names_to_check=None):
    index_list = []
    for index, sample_description in enumerate(sample_names_to_check):
        if name in sample_description and len(name) == len(sample_description):
            print(name)
            index_list.append(index)

    return index_list[:n]


class LibraryLengthNormalizer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Normalise the gene counts to the library size
        X = X.apply(lambda x: (x / (x.sum() if x.sum() != 0 else 1)) * 1000000)
        return X

In [13]:
included_sample_names = [
    "CL-307",
    "CRX-527",
    "Fla-PA",
    "IMDM",
    "LPS",
    "LTA",
    "PGN",
    "Pam3",
    "R848",
]

In [14]:
file_name = "/home/t.afanasyeva/MATseq/notebooks/gene_counts_7128_old_64_train.csv"
data = pd.read_csv(file_name)

labels = data["sample"]
features = data.drop(columns=["sample"])

In [15]:
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=4)
outliers = clf.fit_predict(features)
mask = [outlier != -1 for outlier in outliers]
features = features[mask]
labels = labels[mask]

features_train = features.reset_index(drop=True)
labels_train = labels.reset_index(drop=True)

In [16]:
X_train = features_train

In [17]:
y_train = labels_train

In [18]:
file_name = "/home/t.afanasyeva/MATseq/notebooks/gene_counts_105_test_set.csv"
data = pd.read_csv(file_name)


labels = data["sample"]
features = data.drop(columns=["sample"])

In [19]:
index_list = []
for name in included_sample_names:
    index_list.extend(select_samples(name, n=500, sample_names_to_check=labels))

features_test = features.iloc[index_list]
labels_test = labels.iloc[index_list]

Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
Fla-PA
IMDM
IMDM
IMDM
IMDM
IMDM
IMDM
IMDM
IMDM
IMDM
IMDM
IMDM
IMDM
IMDM
IMDM
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LPS
LTA
LTA
LTA
Pam3
Pam3
Pam3
Pam3
Pam3
Pam3
Pam3
Pam3
Pam3
Pam3
Pam3
Pam3
R848
R848
R848
R848
R848
R848
R848
R848
R848
R848
R848
R848
R848
R848
R848
R848
R848
R848


In [20]:
X_test = features_test

In [21]:
y_test = labels_test

In [22]:
import time
from sklearn.pipeline import Pipeline
from feature_engine.selection import (
    DropDuplicateFeatures,
    DropConstantFeatures,
)
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import PowerTransformer

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel

forest = ExtraTreesClassifier(n_estimators=250, max_depth=5, random_state=1)


feature_transformer = Pipeline(
    [
        (
            "drop_duplicates",
            DropDuplicateFeatures(),
        ),  # insert a transformer that converts everything under 10 to 0
        ("drop_constant_feature", DropConstantFeatures(tol=0.50)),
        ("normalise_for_library_size", LibraryLengthNormalizer()),
        ("cap_outliers", Winsorizer(capping_method="gaussian", tail="right", fold=3)),
    ]
)

feature_selector = Pipeline(
    [
        ("select_k_best", SelectKBest(chi2, k=1000)),
        (
            "trasnform_and_scale",
            PowerTransformer(method="yeo-johnson", standardize=True),
        ),
        ("select_forest", SelectFromModel(forest, max_features=500)),
    ]
)

print("Number of features before: ", X_train.shape[1])
start_time = time.time()


X_train = feature_transformer.fit_transform(X_train)
X_test = feature_transformer.transform(X_test)

feature_selector.fit(X_train, y_train)
X_train = feature_selector.transform(X_train)
X_test = feature_selector.transform(X_test)

end_time = time.time()
print("Number of features after: ", X_train.shape[1])
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time} seconds")

Number of features before:  19992
Number of features after:  322
Elapsed time: 30.643081426620483 seconds


In [23]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [24]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_train.shape[1]
num_classes

9

In [25]:
from sklearn.model_selection import train_test_split

split_params = {
    "test_size": 0.2,
    "random_state": 0,
    "shuffle": True,
}


X_train, X_val, y_train, y_val = train_test_split(
    X_train,
    y_train,
    **split_params,
    stratify=y_train,
)

## 3. Build an architecture


In [37]:
keras.backend.clear_session()
keras.utils.set_random_seed(2)

In [38]:
WANDB_NOTEBOOK_NAME = "20231106 NN fifth sweep"

In [39]:
sweep_config = {
    "method": "bayes",
    "name": "third_sweep",
    "metric": {
        "name": "Validation AUC",
        "goal": "maximize",
    },
    "parameters": {
        "l1": {"values": [16, 24, 32, 40]},
        "l2": {"values": [16, 24, 32, 40]},
        "epochs": {"values": [200, 280, 320]},
        "learning_rate": {"distribution": "uniform", "min": 0.008, "max": 0.02},
        "batch_size": {"values": [12]},
        "weights_limit": {"distribution": "uniform", "min": 0.008, "max": 0.02},
        "dropout_rates": {"values": [0.2, 0.3, 0.4]},
    },
}

In [40]:
sweep_id = wandb.sweep(sweep=sweep_config, project="NN-MATseq")

Create sweep with ID: fnhp0fsv
Sweep URL: https://wandb.ai/afanasyeva-team/NN-MATseq/sweeps/fnhp0fsv


In [41]:
file_path = "/home/t.afanasyeva/MATseq/notebooks/model_zoo"

checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=file_path,
    save_weights_only=False,
    monitor="loss",
    verbose=1,
    save_best_only=True,
    mode="min",
)

In [42]:
default_config = {
    "l1": 100,
    "l2": 80,
    "l3": 27,
    "batch_size": 8,
    "activation": "elu",
    "weights_limit": 0.001,
    "epochs": 400,
    "learning_rate": 0.001,
    "loss": "categorical_crossentropy",
    "optimizer": "adam",
    "dropout_rate1": 0.2,
    "dropout_rate2": 0.5,
    "dropout_rate3": 0.8,
}

In [43]:
sweep_config = {
    "method": "bayes",
    "name": "fifth_sweep",
    "metric": {
        "name": "validation_loss",
        "goal": "minimize",
    },
    "parameters": {
        "l1": {"values": [100, 120, 150]},
        "l2": {"values": [60, 70, 80, 90]},
        "l3": {"values": [18, 27]},
        "learning_rate": {"distribution": "uniform", "min": 0.0001, "max": 0.01},
        "batch_size": {"values": [8, 10, 12]},
        "weights_limit": {"distribution": "uniform", "min": 0.0001, "max": 0.01},
    },
}

In [44]:
from tensorflow.keras.initializers import HeUniform


def train():
    with tf.device("/GPU:0"):
        with wandb.init(project="NN-MATseq", config=default_config) as run:
            batch_size = run.config.batch_size
            activation = run.config.activation
            weights_limit = run.config.weights_limit
            epochs = run.config.epochs
            learning_rate = run.config.learning_rate
            loss = run.config.loss
            l1 = run.config.l1
            l2 = run.config.l2
            l3 = run.config.l3
            dropout_rate1 = run.config.dropout_rate1
            dropout_rate2 = run.config.dropout_rate2
            dropout_rate3 = run.config.dropout_rate3

            if run.config.optimizer == "adam":
                optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
            elif run.config.optimizer == "sgd":
                optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
            elif run.config.optimizer == "rmsprop":
                optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)

            inputs = keras.Input(shape=X_train.shape[1])
            x = keras.layers.UnitNormalization()(inputs)

            x = keras.layers.Dense(
                l1,
                activation=activation,
                kernel_regularizer=keras.regularizers.L2(weights_limit),
                kernel_initializer=HeUniform(),
            )(x)
            x = keras.layers.Dropout(dropout_rate1)(x)

            x = keras.layers.Dense(
                l2,
                activation=activation,
                kernel_regularizer=keras.regularizers.L2(weights_limit),
                kernel_initializer=HeUniform(),
            )(x)
            x = keras.layers.Dropout(dropout_rate2)(x)

            x = keras.layers.Dense(
                l3,
                activation=activation,
                kernel_regularizer=keras.regularizers.L2(weights_limit),
                kernel_initializer=HeUniform(),
            )(x)
            x = keras.layers.Dropout(dropout_rate3)(x)

            outputs = keras.layers.Dense(9, activation="softmax")(x)

            model = keras.Model(
                inputs=inputs,
                outputs=outputs,
                name="MATseq_NN_multiclass_classifier_v0.3",
            )

            model.compile(
                optimizer=optimizer,
                loss=loss,
                metrics=[
                    "accuracy",
                    keras.metrics.AUC(name="auc"),
                ],
            )

            history = model.fit(
                X_train,
                y_train,
                validation_data=(X_val, y_val),
                epochs=epochs,
                batch_size=batch_size,
                verbose=True,
                callbacks=[
                    wandb.keras.WandbCallback(),
                    checkpoint_callback,
                ],
            )

In [45]:
count = 50
wandb.agent(sweep_id, train, count=count)

wandb: Agent Starting Run: 1bflri29 with config:
wandb: 	batch_size: 12
wandb: 	dropout_rates: 0.2
wandb: 	epochs: 200
wandb: 	l1: 32
wandb: 	l2: 16
wandb: 	learning_rate: 0.013635398995136412
wandb: 	weights_limit: 0.01911138273089282
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
cat: /sys/module/amdgpu/initstate: No such file or directory
ERROR:root:Driver not initialized (amdgpu not found in modules)


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/200
1/6 [====>.........................] - ETA: 7s - loss: 5.1857 - accuracy: 0.0833 - auc: 0.4757
Epoch 1: loss improved from inf to 5.17378, saving model to /home/t.afanasyeva/MATseq/notebooks/model_zoo
INFO:tensorflow:Assets written to: /home/t.afanasyeva/MATseq/notebooks/model_zoo/assets


INFO:tensorflow:Assets written to: /home/t.afanasyeva/MATseq/notebooks/model_zoo/assets


6/6 [==============================] - 3s 213ms/step - loss: 5.1738 - accuracy: 0.1406 - auc: 0.5035
Epoch 2/200
1/6 [====>.........................] - ETA: 0s - loss: 4.6765 - accuracy: 0.2500 - auc: 0.6693
Epoch 2: loss improved from 5.17378 to 4.65156, saving model to /home/t.afanasyeva/MATseq/notebooks/model_zoo
INFO:tensorflow:Assets written to: /home/t.afanasyeva/MATseq/notebooks/model_zoo/assets


...

Epoch 1/320
1/6 [====>.........................] - ETA: 6s - loss: 5.0324 - accuracy: 0.0000e+00 - auc: 0.3932
Epoch 1: loss did not improve from 1.07285
6/6 [==============================] - 1s 7ms/step - loss: 5.4207 - accuracy: 0.1250 - auc: 0.4913
Epoch 2/320
1/6 [====>.........................] - ETA: 0s - loss: 4.6649 - accuracy: 0.1667 - auc: 0.6233
Epoch 2: loss did not improve from 1.07285
6/6 [==============================] - 0s 7ms/step - loss: 4.3830 - accuracy: 0.2344 - auc: 0.6325
Epoch 3/320
1/6 [====>.........................] - ETA: 0s - loss: 3.8347 - accuracy: 0.3333 - auc: 0.7183
Epoch 3: loss did not improve from 1.07285
6/6 [==============================] - 0s 6ms/step - loss: 3.5540 - accuracy: 0.3438 - auc: 0.7410
Epoch 4/320
1/6 [====>.........................] - ETA: 0s - loss: 3.0278 - accuracy: 0.5833 - auc: 0.8082
Epoch 4: loss did not improve from 1.07285
6/6 [==============================] - 0s 7ms/step - loss: 3.1526 - accuracy: 0.3906 - auc: 0.7731


wandb: Ctrl + C detected. Stopping sweep.


Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7fbd1c9a9150>> (for post_run_cell), with arguments args (<ExecutionResult object at 7fbd241980d0, execution_count=45 error_before_exec=None error_in_exec=None info=<ExecutionInfo object at 7fbd24198430, raw_cell="count = 50
wandb.agent(sweep_id, train, count=coun.." store_history=True silent=False shell_futures=True cell_id=vscode-notebook-cell:Untitled-1.ipynb?jupyter-notebook#X34sdW50aXRsZWQ%3D> result=None>,),kwargs {}:

Epoch 221: loss did not improve from 1.07285
6/6 [==============================] - 0s 10ms/step - loss: 1.4811 - accuracy: 0.6094 - auc: 0.9371
Epoch 222/320
1/6 [====>.........................] - ETA: 0s - loss: 1.3024 - accuracy: 0.5833 - auc: 0.9531
Epoch 222: loss did not improve from 1.07285
6/6 [==============================] - 0s 7ms/step - loss: 1.4708 - accuracy: 0.6094 - auc: 0.9352
Epoch 223/320
1/6 [====>.........................] - ETA: 0s - loss:

Traceback (most recent call last):


TypeError: _WandbInit._pause_backend() takes 1 positional argument but 2 were given